<a href="https://colab.research.google.com/github/camily-a/SI-not-2021/blob/main/SistemaPerfilSocioeconimico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

----- INSTRUÇÕES ----------
- Insira o documento de dados (do tipo .xlsx) na área de arquivos ao lado esquerdo.

- Clique no botão de executar código.

- Insira o nome do arquivo adicionado(com .xlsx).


In [10]:
import pandas as pd #apelido da biblioteca

name = input('Insira o nome do arquivo de dados (inclui .xlsx) : ')
tabela = pd.read_excel(name) #importa dados 

################

tabela = tabela.sort_values(by='Carimbo de data/hora', ascending=True) # organiza tabela por data de envio
tabela = tabela.drop_duplicates(subset=["Informe os 7 últimos dígitos do seu RA (109048xxxxxxx) "], keep="last") #exclui linhas duplicadas mantendo a última enviada

# exclui colunas irrelevantes para a Análise
tabela = tabela.drop("Carimbo de data/hora", axis=1) # axis 1 p/ coluna e 0 para linha
tabela = tabela.drop("Informe os 7 últimos dígitos do seu RA (109048xxxxxxx) ", axis=1)

# mostra todas as linhas da tabela
pd.set_option("max_rows", None)

########### 


# padronizando valores iguais , escritos de forma diferente (substituindo)

# pessoas no domicilio

tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"] = tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].replace(regex=['Quatro', '4 pessoas'], value=4)
tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"] = tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].replace(regex='03', value='3')




#################

# Descobrindo idade a partir da data de nascimento

from datetime import date 

data_nasc = (tabela["Qual a sua data de nascimento?"])
days_in_year = 365.25 
agora = pd.Timestamp('today')
 
idade = (agora - data_nasc).dt.days / (days_in_year)
idade = idade.astype(int)

tabela["Qual a sua data de nascimento?"] = idade
tabela.rename(columns={"Qual a sua data de nascimento?": 'Idade'}, inplace = True)

############

# agrupando idades
import sys

tabela['Idade']=pd.cut(
    tabela['Idade'],
    bins=[-1, 20, 30, 41, sys.maxsize],
    labels=['Até 20 anos', 'Entre 21 e 30 anos', 'Entre 31 e 40 anos', 'Mais de 40 anos']
 )

tabela.rename(columns={"Idade": 'Faixa Etária'}, inplace = True)

display(tabela)


#################



#Criando os gráficos

import plotly.express as px # importa a biblioteca

# loop faz todos os gráficos por coluna da tabela de dados
for coluna in tabela:
  qtd = tabela[coluna].value_counts()  #conta os dados da coluna
    # faz o gráfico de pizza usando os dados contados
  grafico = px.pie(names = qtd.index,
                    values = qtd.values,
            title=coluna, color_discrete_sequence=px.colors.sequential.RdBu) 
  # algumas formatações do gráfico
  grafico.update_traces(hovertemplate = "%{label} <br>N° de Alunos: %{value}", textfont_size=20,
              title_font_color='#000000', title_font_size=20)
  # mostra gráfico
  display(grafico)
  


Insira o nome do arquivo de dados (inclui .xlsx) : Resultados.xlsx


,Qual o seu curso?,Qual o período que cursa?,Qual o estado do Brasil que você nasceu?,Qual sua cidade de residência?,Qual o seu gênero?,Faixa Etária,Qual é o seu estado civil?,"Você é portador de alguma deficiência? (Pode selecionar mais de uma, se for o caso)",Quantos filhos você tem?,Com quem você mora atualmente?,"Quantas pessoas, incluindo você, moram no seu domicílio?",Qual a situação do domicílio onde mora?,Tempo de moradia neste domicílio.,Qual a faixa de renda mensal da sua família (em Salários Mínimos)?,Quantos de cada um dos itens abaixo há em seu domicílio? [Televisor:],Quantos de cada um dos itens abaixo há em seu domicílio? [Video cassete e(ou) DVD:],Quantos de cada um dos itens abaixo há em seu domicílio? [Rádio:],Quantos de cada um dos itens abaixo há em seu domicílio? [Automóvel: ],Quantos de cada um dos itens abaixo há em seu domicílio? [Motocicleta:],Quantos de cada um dos itens abaixo há em seu domicílio? [Máquina de lavar roupa e(ou) Tanquinho:],Quantos de cada um dos itens abaixo há em seu domicílio? [Geladeira: ],Quantos de cada um dos itens abaixo há em seu domicílio? [Celular comum: ],Quantos de cada um dos itens abaixo há em seu domicílio? [Smartphone:],Quantos de cada um dos itens abaixo há em seu domicílio? [Microcomputador de mesa:],Quantos de cada um dos itens abaixo há em seu domicílio? [Notebook: ],No seu domicílio você tem: [Telefone fixo:],No seu domicílio você tem: [Internet:],No seu domicílio você tem: [TV por assinatura:],No seu domicílio você tem: [Empregada mensalista:],Você trabalha?,Qual o seu vínculo com o emprego?,Qual a área do seu trabalho?,Qual seu horário de trabalho?,Qual a empresa que você está contratado agora?,Você tem plano de saúde privado?,Qual o grau de escolaridade do seu pai?,Qual o grau de escolaridade da sua mãe?,Na sua vida escolar você estudou...,Com que frequência você utiliza microcomputadores?,Onde você utiliza microcomputadores? [Em casa:],...,Onde você utiliza microcomputadores? [Na escola:],Onde você utiliza microcomputadores? [Em outros lugares:],Com qual finalidade você utiliza microcomputadores? [Para trabalhos profissionais: ],Com qual finalidade você utiliza microcomputadores? [Para trabalhos escolares:],"Com qual finalidade você utiliza microcomputadores? [Para entretenimento (músicas, vídeos, redes sociais, etc): ]",Com qual finalidade você utiliza microcomputadores? [Para comunicação por e-mail: ],Com qual finalidade você utiliza microcomputadores? [Para operações bancárias: ],Com qual finalidade você utiliza microcomputadores? [Para compras eletrônicas: ],Como você classifica seu conhecimento em informática?,Qual o seu conhecimento em relação aos aplicativos a seguir? [Windows:],Qual o seu conhecimento em relação aos aplicativos a seguir? [Linux:],"Qual o seu conhecimento em relação aos aplicativos a seguir? [Editores de textos (Word, Writer, etc.):]","Qual o seu conhecimento em relação aos aplicativos a seguir? [Planilhas eletrônicas (Excel, Calc, etc.):]","Qual o seu conhecimento em relação aos aplicativos a seguir? [Apresentadores (Powerpoint, Impress, Prezzi, etc.):]",Qual o seu conhecimento em relação aos aplicativos a seguir? [Sistemas de Gestão Empresarial:],Inglês,Espanhol,Outro idiomas,Considere a busca por informação nos seguintes meios de comunicação: [TV],Considere a busca por informação nos seguintes meios de comunicação: [Internet],Considere a busca por informação nos seguintes meios de comunicação: [Revistas],Considere a busca por informação nos seguintes meios de comunicação: [Jornais],Considere a busca por informação nos seguintes meios de comunicação: [Rádio],Considere a busca por informação nos seguintes meios de comunicação: [Redes sociais],Considere a busca por informação nos seguintes meios de comunicação: [Conversas com Amigos],"Se você lê jornal, qual a frequência?","Se você lê jornal, quais os assuntos que mais lê?","Não considerando os livros escolares, quantos livros você lê por ano (em média)?","Se você lê livros literários, 